Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
!if test -d /workspaces/tortoise-tts; then git clone https://github.com/jnordberg/tortoise-tts.git; fi
!pip3 install transformers==4.19.0
!pip3 install -r requirements.txt

!pip3 install einops==0.5.0
!pip3 install rotary_embedding_torch==0.1.5
!pip3 install unidecode==1.3.5

In [8]:
!apt -y install youtube-dl ffmpeg
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
youtube-dl is already the newest version (2020.03.24-1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [19]:
%cd /workspaces/tortoise-tts
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

/content/tortoise-tts


100% (1716988501 of 1716988501) |########| Elapsed Time: 0:00:06 Time:  0:00:06


Done.


100% (60938957 of 60938957) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [22]:
# This is the text that will be spoken.
text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [ ]:
%cd /content/tortoise-tts/tortoise/voices/gec



In [ ]:
%cd /content/tortoise-tts/tortoise/voices/gec

### Random shit to fix random shit
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
### End random shit

yt_clips = [ {
        'url': 'https://www.youtube.com/watch?v=DX5ez4-ftes',
        'start': '00:00:06',
        'time': '00:00:20',
        'name': 'dog-food'
        },
        { 
        'name': 'mallet-acapella',
        'url': 'https://www.youtube.com/watch?v=DX5ez4-ftes',
        'start': '00:00:06',
        'time': '00:00:20'
        }
        ]
  # download it from yt, convert, clip, clean up
for i, yt_clip_info in enumerate(yt_clips):
  url = yt_clip_info['url']
  name = yt_clip_info['name']
  !test -f {i+1}full.m4a || yt-dlp -x "$url" --format mp4 -o {i+1}full
  # convert to mp3
  !test -f {i+1}full.mp3 || ffmpeg -i {i+1}full.m4a -c:v copy -c:a libmp3lame -q:a 4 {i+1}full.mp3 
  !ffmpeg -ss 00:00:15 -t 00:00:30.00 -i {i+1}full.mp3 {i+1}.mp3  # clip it
  !rm {i+1}full.m4a


/content/tortoise-tts/tortoise/voices/gec
[youtube] Extracting URL: https://www.youtube.com/watch?v=DX5ez4-ftes
[youtube] DX5ez4-ftes: Downloading webpage
[youtube] DX5ez4-ftes: Downloading android player API JSON
[info] DX5ez4-ftes: Downloading 1 format(s): 22
[download] Destination: 1full
[download] 100% of    4.53MiB in 00:00:00 at 54.16MiB/s
[ExtractAudio] Destination: 1full.m4a
Deleting original file 1full (pass -k to keep)
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --en

In [24]:
%cd /content/tortoise-tts/tortoise/voices/gec
for i in range(len(yt_clips)):
  !ffmpeg -ss 00:00:15 -t 00:00:30.00 -i 1gec.mp3 1.mp3

/content/tortoise-tts/tortoise/voices/gec


NotImplementedError: ignored

In [23]:
%cd /content/tortoise-tts/
# Pick one of the voices from the output above
voice = 'gec'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

/content/tortoise-tts
Generating autoregressive samples..


100%|██████████| 6/6 [00:56<00:00,  9.41s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 6/6 [00:11<00:00,  1.84s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:21<00:00,  3.76it/s]


In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')